# Train ML model to correct predictions of week 3-4 & 5-6

This notebook create a Machine Learning `ML_model` to predict weeks 3-4 & 5-6 based on `S2S` weeks 3-4 & 5-6 forecasts and is compared to `CPC` observations for the [`s2s-ai-challenge`](https://s2s-ai-challenge.github.io/).

# Synopsis

## Method: `ML-based mean bias reduction`

- calculate the ML-based bias from 2000-2019 deterministic ensemble mean forecast
- remove that the ML-based bias from 2020 forecast deterministic ensemble mean forecast

## Data used

type: renku datasets

Training-input for Machine Learning model:
- hindcasts of models:
    - ECMWF: `ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr`

Forecast-input for Machine Learning model:
- real-time 2020 forecasts of models:
    - ECMWF: `ecmwf_forecast-input_2020_biweekly_deterministic.zarr`

Compare Machine Learning model forecast against against ground truth:
- `CPC` observations:
    - `hindcast-like-observations_biweekly_deterministic.zarr`
    - `forecast-like-observations_2020_biweekly_deterministic.zarr`

## Resources used
for training, details in reproducibility

- platform: renku
- memory: 8 GB
- processors: 2 CPU
- storage required: 10 GB

## Safeguards

All points have to be [x] checked. If not, your submission is invalid.

Changes to the code after submissions are not possible, as the `commit` before the `tag` will be reviewed.
(Only in exceptions and if previous effort in reproducibility can be found, it may be allowed to improve readability and reproducibility after November 1st 2021.)

### Safeguards to prevent [overfitting](https://en.wikipedia.org/wiki/Overfitting?wprov=sfti1) 

If the organizers suspect overfitting, your contribution can be disqualified.

  - [x] We did not use 2020 observations in training (explicit overfitting and cheating)
  - [x] We did not repeatedly verify my model on 2020 observations and incrementally improved my RPSS (implicit overfitting)
  - [x] We provide RPSS scores for the training period with script `print_RPS_per_year`, see in section 6.3 `predict`.
  - [x] We tried our best to prevent [data leakage](https://en.wikipedia.org/wiki/Leakage_(machine_learning)?wprov=sfti1).
  - [x] We honor the `train-validate-test` [split principle](https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets). This means that the hindcast data is split into `train` and `validate`, whereas `test` is withheld.
  - [x] We did not use `test` explicitly in training or implicitly in incrementally adjusting parameters.
  - [x] We considered [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)).

### Safeguards for Reproducibility
Notebook/code must be independently reproducible from scratch by the organizers (after the competition), if not possible: no prize
  - [x] All training data is publicly available (no pre-trained private neural networks, as they are not reproducible for us)
  - [x] Code is well documented, readable and reproducible.
  - [x] Code to reproduce training and predictions is preferred to run within a day on the described architecture. If the training takes longer than a day, please justify why this is needed. Please do not submit training piplelines, which take weeks to train.

# Todos to improve template

This is just a demo.

- [ ] use multiple predictor variables and two predicted variables
- [ ] for both `lead_time`s in one go
- [ ] consider seasonality, for now all `forecast_time` months are mixed
- [ ] make probabilistic predictions with `category` dim, for now works deterministic

# Imports

In [1]:
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt

import xarray as xr
xr.set_options(display_style='text')
import numpy as np

from dask.utils import format_bytes
import xskillscore as xs

# Get training data

preprocessing of input data may be done in separate notebook/script

## Hindcast

get weekly initialized hindcasts

In [2]:
v='t2m'

In [3]:
# preprocessed as renku dataset
!renku storage pull ../data/ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr

In [3]:
hind_2000_2019 = xr.open_zarr("../data/ecmwf_hindcast-input_2000-2019_biweekly_deterministic.zarr", consolidated=True)

In [5]:
# preprocessed as renku dataset
!renku storage pull ../data/ecmwf_forecast-input_2020_biweekly_deterministic.zarr

In [4]:
fct_2020 = xr.open_zarr("../data/ecmwf_forecast-input_2020_biweekly_deterministic.zarr", consolidated=True)

## Observations
corresponding to hindcasts

In [7]:
# preprocessed as renku dataset
!renku storage pull ../data/hindcast-like-observations_2000-2019_biweekly_deterministic.zarr

In [5]:
obs_2000_2019 = xr.open_zarr("../data/hindcast-like-observations_2000-2019_biweekly_deterministic.zarr", consolidated=True)#[v]

In [9]:
# preprocessed as renku dataset
!renku storage pull ../data/forecast-like-observations_2020_biweekly_deterministic.zarr

In [6]:
obs_2020 = xr.open_zarr("../data/forecast-like-observations_2020_biweekly_deterministic.zarr", consolidated=True)#[v]

# ML model

based on [Weatherbench](https://github.com/pangeo-data/WeatherBench/blob/master/quickstart.ipynb)

In [ ]:
# run once only and dont commit
!git clone https://github.com/pangeo-data/WeatherBench/

In [7]:
import sys
sys.path.insert(1, 'WeatherBench')
from WeatherBench.src.train_nn import DataGenerator, PeriodicConv2D, create_predictions
import tensorflow.keras as keras

In [8]:
bs=32

import numpy as np
class DataGenerator(keras.utils.Sequence):
    def __init__(self, fct, verif, lead_time, batch_size=bs, shuffle=True, load=True,
                 mean=None, std=None):
        """
        Data generator for WeatherBench data.
        Template from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

        Args:
            fct: forecasts from S2S models: xr.DataArray (xr.Dataset doesnt work properly)
            verif: observations with same dimensionality (xr.Dataset doesnt work properly)
            lead_time: Lead_time as in model
            batch_size: Batch size
            shuffle: bool. If True, data is shuffled.
            load: bool. If True, datadet is loaded into RAM.
            mean: If None, compute mean from data.
            std: If None, compute standard deviation from data.
            
        Todo:
        - use number in a better way, now uses only ensemble mean forecast
        - dont use .sel(lead_time=lead_time) to train over all lead_time at once
        - be sensitive with forecast_time, pool a few around the weekofyear given
        - use more variables as predictors
        - predict more variables
        """

        if isinstance(fct, xr.Dataset):
            print('convert fct to array')
            fct = fct.to_array().transpose(...,'variable')
            self.fct_dataset=True
        else:
            self.fct_dataset=False
            
        if isinstance(verif, xr.Dataset):
            print('convert verif to array')
            verif = verif.to_array().transpose(...,'variable')
            self.verif_dataset=True
        else:
            self.verif_dataset=False
        
        #self.fct = fct
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.lead_time = lead_time

        self.fct_data = fct.transpose('forecast_time', ...).sel(lead_time=lead_time)
        self.fct_mean = self.fct_data.mean('forecast_time').compute() if mean is None else mean
        self.fct_std = self.fct_data.std('forecast_time').compute() if std is None else std
        
        self.verif_data = verif.transpose('forecast_time', ...).sel(lead_time=lead_time)
        self.verif_mean = self.verif_data.mean('forecast_time').compute() if mean is None else mean
        self.verif_std = self.verif_data.std('forecast_time').compute() if std is None else std

        # Normalize
        self.fct_data = (self.fct_data - self.fct_mean) / self.fct_std
        self.verif_data = (self.verif_data - self.verif_mean) / self.verif_std
        
        self.n_samples = self.fct_data.forecast_time.size
        self.forecast_time = self.fct_data.forecast_time

        self.on_epoch_end()

        # For some weird reason calling .load() earlier messes up the mean and std computations
        if load:
            # print('Loading data into RAM')
            self.fct_data.load()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(self.n_samples / self.batch_size))

    def __getitem__(self, i):
        'Generate one batch of data'
        idxs = self.idxs[i * self.batch_size:(i + 1) * self.batch_size]
        # got all nan if nans not masked
        X = self.fct_data.isel(forecast_time=idxs).fillna(0.).values
        y = self.verif_data.isel(forecast_time=idxs).fillna(0.).values
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.idxs = np.arange(self.n_samples)
        if self.shuffle == True:
            np.random.shuffle(self.idxs)

In [9]:
# 2 bi-weekly `lead_time`: week 3-4
lead = hind_2000_2019.isel(lead_time=0).lead_time

lead

<xarray.DataArray 'lead_time' ()>
array(1209600000000000, dtype='timedelta64[ns]')
Coordinates:
    lead_time  timedelta64[ns] 14 days
Attributes:
    aggregate:      The pd.Timedelta corresponds to the first day of a biweek...
    description:    Forecast period is the time interval between the forecast...
    long_name:      lead time
    standard_name:  forecast_period
    week34_t2m:     mean[14 days, 27 days]
    week34_tp:      28 days minus 14 days
    week56_t2m:     mean[28 days, 41 days]
    week56_tp:      42 days minus 28 days

In [10]:
# mask, needed?
hind_2000_2019 = hind_2000_2019.where(obs_2000_2019.isel(forecast_time=0, lead_time=0,drop=True).notnull())

## data prep: train, valid, test

[Use the hindcast period to split train and valid.](https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets) Do not use the 2020 data for testing!

In [11]:
# time is the forecast_time
time_train_start,time_train_end='2000','2017' # train
time_valid_start,time_valid_end='2018','2019' # valid
time_test = '2020'                            # test

In [12]:
dg_train = DataGenerator(
    hind_2000_2019.mean('realization').sel(forecast_time=slice(time_train_start,time_train_end))[v],
    obs_2000_2019.sel(forecast_time=slice(time_train_start,time_train_end))[v],
    lead_time=lead, batch_size=bs, load=True)

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [13]:
dg_valid = DataGenerator(
    hind_2000_2019.mean('realization').sel(forecast_time=slice(time_valid_start,time_valid_end))[v],
    obs_2000_2019.sel(forecast_time=slice(time_valid_start,time_valid_end))[v],
    lead_time=lead, batch_size=bs, shuffle=False, load=True)

/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
/work/mh0727/m300524/conda-envs/s2s-ai/lib/python3.7/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


In [14]:
# do not use, delete?
dg_test = DataGenerator(
    fct_2020.mean('realization').sel(forecast_time=time_test)[v],
    obs_2020.sel(forecast_time=time_test)[v],
    lead_time=lead, batch_size=bs, load=True, mean=dg_train.fct_mean, std=dg_train.fct_std, shuffle=False)

In [15]:
X, y = dg_valid[0]
X.shape, y.shape

((32, 121, 240), (32, 121, 240))

In [16]:
# short look into training data: large biases
# any problem from normalizing?
# i=4
# xr.DataArray(np.vstack([X[i],y[i]])).plot(yincrease=False, robust=True)

## `fit`

In [17]:
cnn = keras.models.Sequential([
    PeriodicConv2D(filters=32, kernel_size=5, conv_kwargs={'activation':'relu'}, input_shape=(32, 64, 1)),
    PeriodicConv2D(filters=1, kernel_size=5)
])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [18]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
periodic_conv2d (PeriodicCon (None, 32, 64, 32)        832       
_________________________________________________________________
periodic_conv2d_1 (PeriodicC (None, 32, 64, 1)         801       
Total params: 1,633
Trainable params: 1,633
Non-trainable params: 0
_________________________________________________________________


In [19]:
cnn.compile(keras.optimizers.Adam(1e-4), 'mse')

In [20]:
import warnings
warnings.simplefilter("ignore")

In [21]:
cnn.fit(dg_train, epochs=2, validation_data=dg_valid)

Epoch 1/2
30/30 [==============================] - 20s 638ms/step - loss: 0.2237 - val_loss: 0.1192
Epoch 2/2
30/30 [==============================] - 18s 594ms/step - loss: 0.1046 - val_loss: 0.0647


## `predict`

Create predictions and print `mean(variable, lead_time, longitude, weighted latitude)` RPSS for all years as calculated by `skill_by_year`.

In [22]:
from scripts import add_valid_time_from_forecast_reference_time_and_lead_time

def _create_predictions(model, dg, lead):
    """Create non-iterative predictions"""
    preds = model.predict(dg).squeeze()
    # Unnormalize
    preds = preds * dg.fct_std.values + dg.fct_mean.values
    if dg.verif_dataset:
        da = xr.DataArray(
                    preds,
                    dims=['forecast_time', 'latitude', 'longitude','variable'],
                    coords={'forecast_time': dg.fct_data.forecast_time, 'latitude': dg.fct_data.latitude,
                            'longitude': dg.fct_data.longitude},
                ).to_dataset() # doesnt work yet
    else:
        da = xr.DataArray(
                    preds,
                    dims=['forecast_time', 'latitude', 'longitude'],
                    coords={'forecast_time': dg.fct_data.forecast_time, 'latitude': dg.fct_data.latitude,
                            'longitude': dg.fct_data.longitude},
                )
    da = da.assign_coords(lead_time=lead)
    # da = add_valid_time_from_forecast_reference_time_and_lead_time(da)
    return da

Magics library could not be found


In [23]:
# optionally masking the ocean when making probabilistic
mask = obs_2020.std(['lead_time','forecast_time']).notnull()

In [24]:
from scripts import make_probabilistic

In [30]:
!renku storage pull ../data/hindcast-like-observations_2000-2019_biweekly_tercile-edges.nc

In [25]:
cache_path='../data'
tercile_file = f'{cache_path}/hindcast-like-observations_2000-2019_biweekly_tercile-edges.nc'
tercile_edges = xr.open_dataset(tercile_file)

In [26]:
# this is not useful but results have expected dimensions
# actually train for each lead_time

def create_predictions(cnn, fct, obs, time):
    preds_test=[]
    for lead in fct.lead_time:
        dg = DataGenerator(fct.mean('realization').sel(forecast_time=time)[v],
                           obs.sel(forecast_time=time)[v],
                           lead_time=lead, batch_size=bs, mean=dg_train.fct_mean, std=dg_train.fct_std, shuffle=False)
        preds_test.append(_create_predictions(cnn, dg, lead))
    preds_test = xr.concat(preds_test, 'lead_time')
    preds_test['lead_time'] = fct.lead_time
    # add valid_time coord
    preds_test = add_valid_time_from_forecast_reference_time_and_lead_time(preds_test)
    preds_test = preds_test.to_dataset(name=v)
    # add fake var
    preds_test['tp'] = preds_test['t2m']
    # make probabilistic
    preds_test = make_probabilistic(preds_test.expand_dims('realization'), tercile_edges, mask=mask)
    return preds_test

### `predict` training period in-sample

In [33]:
!renku storage pull ../data/forecast-like-observations_2020_biweekly_terciled.nc

In [34]:
!renku storage pull ../data/hindcast-like-observations_2000-2019_biweekly_terciled.zarr

In [27]:
from scripts import skill_by_year
import os
if os.environ['HOME'] == '/home/jovyan':
    import pandas as pd
    # assume on renku with small memory
    step = 2
    skill_list = []
    for year in np.arange(int(time_train_start), int(time_train_end) -1, step): # loop over years to consume less memory on renku
        preds_is = create_predictions(cnn, hind_2000_2019, obs_2000_2019, time=slice(str(year), str(year+step-1))).compute()
        skill_list.append(skill_by_year(preds_is))
    skill = pd.concat(skill_list)
else: # with larger memory, simply do
    preds_is = create_predictions(cnn, hind_2000_2019, obs_2000_2019, time=slice(time_train_start, time_train_end))
    skill = skill_by_year(preds_is)
skill

,RPSS
year,
2000,-1.233510
2001,-1.388854
2002,-1.465850
2003,-1.413647
2004,-1.414394
2005,-1.543710
2006,-1.440741
2007,-1.544764
2008,-1.432690


### `predict` validation period out-of-sample

In [28]:
preds_os = create_predictions(cnn, hind_2000_2019, obs_2000_2019, time=slice(time_valid_start, time_valid_end))

skill_by_year(preds_os)

,RPSS
year,
2018,-1.476327
2019,-1.529335


### `predict` test

In [29]:
preds_test = create_predictions(cnn, fct_2020, obs_2020, time=time_test)

skill_by_year(preds_test)

,RPSS
year,
2020,-1.443918


# Submission

In [30]:
from scripts import assert_predictions_2020
assert_predictions_2020(preds_test)

In [32]:
preds_test.to_netcdf('../submissions/ML_prediction_2020.nc')

In [ ]:
# !git add ../submissions/ML_prediction_2020.nc
# !git add ML_train_and_prediction.ipynb

In [ ]:
# !git commit -m "template_test commit message" # whatever message you want

In [ ]:
# !git tag "submission-template_test-0.0.1" # if this is to be checked by scorer, only the last submitted==tagged version will be considered

In [ ]:
# !git push --tags

# Reproducibility

## memory

In [48]:
# https://phoenixnap.com/kb/linux-commands-check-memory-usage
!free -g

              total        used        free      shared  buff/cache   available
Mem:             47          10          24           0          11          36
Swap:             0           0           0


## CPU

In [49]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   40 bits physical, 48 bits virtual
CPU(s):                          16
On-line CPU(s) list:             0-15
Thread(s) per core:              1
Core(s) per socket:              1
Socket(s):                       16
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           61
Model name:                      Intel Core Processor (Broadwell, IBRS)
Stepping:                        2
CPU MHz:                         2194.916
BogoMIPS:                        4389.83
Virtualization:                  VT-x
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       512 KiB
L1i cache:                       512 KiB
L2 cache:                        64 MiB
NUMA node0 CPU(s):               0-15
Vul

## software

In [50]:
!conda list

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       1_gnu    conda-forge
_pytorch_select           0.1                       cpu_0    defaults
_tflow_select             2.3.0                       mkl    defaults
absl-py                   0.12.0           py38h06a4308_0    defaults
aiobotocore               1.2.2              pyhd3eb1b0_0    defaults
aiohttp                   3.7.4.post0              pypi_0    pypi
aioitertools              0.7.1              pyhd3eb1b0_0    defaults
alembic                   1.4.3              pyh9f0ad1d_0    conda-forge
ansiwrap                  0.8.4                    pypi_0    pypi
appdirs                   1.4.4                    pypi_0    pypi
argcomplete               1.12.2                   pypi_0    pypi
argon2-cffi               20.1.0           py38h497a2fe_2